In [29]:
# Cargamos las librerias
import ast
import pandas as pd
import numpy as np
import json

#Como el se que mi Dataset tiene muchas columnas seteo a Pandas para que me muestre todas
pd.set_option('display.max_columns', None)

# Se carga el archivo CSV dentro de un DataFrame de Pandas

df = pd.read_csv('movies_dataset.csv')



C:\Users\usuario\AppData\Local\Temp\ipykernel_13380\2854674742.py:12: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_dataset.csv')


In [30]:
#Realizo una exploración inicial de los datos
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [31]:
# Como observamos algunas columnas tienen diccionarios empaquetados en formato de listas "[]", procedo a eliminar los corchetes.

df['belongs_to_collection'] = df['belongs_to_collection'].astype(str).str.replace('[','').str.replace(']','')
df['genres'] = df['genres'].astype(str).str.replace('[','').str.replace(']','')
df['spoken_languages'] = df['spoken_languages'].astype(str).str.replace('[','').str.replace(']','')
df['production_companies'] = df['production_companies'].astype(str).str.replace('[','').str.replace(']','')
df['production_countries'] = df['production_countries'].astype(str).str.replace('[','').str.replace(']','')

df.head(2)

C:\Users\usuario\AppData\Local\Temp\ipykernel_13380\1159837356.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['belongs_to_collection'] = df['belongs_to_collection'].astype(str).str.replace('[','').str.replace(']','')
C:\Users\usuario\AppData\Local\Temp\ipykernel_13380\1159837356.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['genres'] = df['genres'].astype(str).str.replace('[','').str.replace(']','')
C:\Users\usuario\AppData\Local\Temp\ipykernel_13380\1159837356.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"{'id': 16, 'name': 'Animation'}, {'id': 35, 'n...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"{'name': 'Pixar Animation Studios', 'id': 3}","{'iso_3166_1': 'US', 'name': 'United States of...",1995-10-30,373554033.0,81.0,"{'iso_639_1': 'en', 'name': 'English'}",Released,NaN,Toy Story,False,7.7,5415.0
1,False,nan,65000000,"{'id': 12, 'name': 'Adventure'}, {'id': 14, 'n...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"{'name': 'TriStar Pictures', 'id': 559}, {'nam...","{'iso_3166_1': 'US', 'name': 'United States of...",1995-12-15,262797249.0,104.0,"{'iso_639_1': 'en', 'name': 'English'}, {'iso_...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [32]:
# Creo una funcion que me permita estandarizar los datos que se encuentran como diccionario en diferentes columnas a formato JSON, para posteriormente desempaquetarlos.
# En formato JSON las claves y valor van entre comillas dobles
# En esta misma funcion aprovecho para que si encuentra valores faltantes devuelva 0


def fix_json(x):
    try:
        d = json.loads(x.replace("'", "\""))
        return json.dumps({str(k): str(v) for k,v in d.items()})
    except:
        return 0



In [33]:
#Aplico la funcion a las columnas que me intereza estandarizar

df['belongs_to_collection'] = df['belongs_to_collection'].apply(fix_json)
df['genres'] = df['genres'].apply(fix_json)
df['spoken_languages'] = df['spoken_languages'].apply(fix_json)
df['production_companies'] = df['production_companies'].apply(fix_json)
df['production_countries'] = df['production_countries'].apply(fix_json)

df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{""id"": ""10194"", ""name"": ""Toy Story Collection""...",30000000,0,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"{""name"": ""Pixar Animation Studios"", ""id"": ""3""}","{""iso_3166_1"": ""US"", ""name"": ""United States of...",1995-10-30,373554033.0,81.0,"{""iso_639_1"": ""en"", ""name"": ""English""}",Released,NaN,Toy Story,False,7.7,5415.0
1,False,0,65000000,0,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,0,"{""iso_3166_1"": ""US"", ""name"": ""United States of...",1995-12-15,262797249.0,104.0,0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [34]:
from pandas import json_normalize

# Aplico un funcion lambda y convierto los datos de columnas que tienen formato JSON a un diccionario, comprobando si el valor esta en formato de string, si no lo esta los empaqueta en un diccionario vaico
df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x: json.loads(x) if isinstance(x, str) else {})
df['genres'] = df['genres'].apply(lambda x: json.loads(x) if isinstance(x, str) else {})
df['spoken_languages'] = df['spoken_languages'].apply(lambda x: json.loads(x) if isinstance(x, str) else {})
df['production_companies'] = df['production_companies'].apply(lambda x: json.loads(x) if isinstance(x, str) else {})
df['production_countries'] = df['production_countries'].apply(lambda x: json.loads(x) if isinstance(x, str) else {})

# Desanido los datos y agregar nuevas columnas al DataFrame
df = pd.concat([df.drop(['belongs_to_collection'], axis=1),
                json_normalize(df['belongs_to_collection']).add_prefix('belongs_to_collection_')],
               axis=1)
df = pd.concat([df.drop(['genres'], axis=1),
                json_normalize(df['genres']).add_prefix('genres_')],
               axis=1)
df = pd.concat([df.drop(['spoken_languages'], axis=1),
                json_normalize(df['spoken_languages']).add_prefix('spoken_languages_')],
               axis=1)
df = pd.concat([df.drop(['production_companies'], axis=1),
                json_normalize(df['production_companies']).add_prefix('production_companies_')],
               axis=1)
df = pd.concat([df.drop(['production_countries'], axis=1),
                json_normalize(df['production_countries']).add_prefix('production_countries_')],
               axis=1)


df.head(2)

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,belongs_to_collection_id,belongs_to_collection_name,belongs_to_collection_poster_path,belongs_to_collection_backdrop_path,genres_id,genres_name,spoken_languages_iso_639_1,spoken_languages_name,production_companies_name,production_companies_id,production_countries_iso_3166_1,production_countries_name
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,NaN,NaN,en,English,Pixar Animation Studios,3,US,United States of America
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,United States of America


In [35]:
# Reviso que columnas tienen valores faltantes y su cantidad
df.isnull().sum()

adult                                      0
budget                                     0
homepage                               37684
id                                         0
imdb_id                                   17
original_language                         11
original_title                             0
overview                                 954
popularity                                 5
poster_path                              386
release_date                              87
revenue                                    6
runtime                                  263
status                                    87
tagline                                25054
title                                      6
video                                      6
vote_average                               6
vote_count                                 6
belongs_to_collection_id               42298
belongs_to_collection_name             42298
belongs_to_collection_poster_path      42298
belongs_to

In [36]:
# Remplazo los valores faltantes en las columnas solicitadas por 0
df['revenue'].fillna(0, inplace=True)
df['budget'].fillna(0, inplace=True)

In [37]:
# Convierto la columna a formato fecha
df["release_date"]= pd.to_datetime(df['release_date'], errors= "coerce").dt.strftime('%Y-%m-%d')

In [38]:
# Elimino las filas que tienen valor faltante en la columna release_date
df = df.dropna(subset=['release_date'])
df.isnull().sum()

adult                                      0
budget                                     0
homepage                               37610
id                                         0
imdb_id                                   14
original_language                         11
original_title                             0
overview                                 941
popularity                                 0
poster_path                              339
release_date                               0
revenue                                    0
runtime                                  246
status                                    80
tagline                                24978
title                                      0
video                                      0
vote_average                               0
vote_count                                 0
belongs_to_collection_id               42211
belongs_to_collection_name             42211
belongs_to_collection_poster_path      42211
belongs_to

In [39]:
# Creao la columna release_year
df['release_year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year

In [40]:
    
df.head()

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,belongs_to_collection_id,belongs_to_collection_name,belongs_to_collection_poster_path,belongs_to_collection_backdrop_path,genres_id,genres_name,spoken_languages_iso_639_1,spoken_languages_name,production_companies_name,production_companies_id,production_countries_iso_3166_1,production_countries_name,release_year
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,NaN,NaN,en,English,Pixar Animation Studios,3,US,United States of America,1995
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,United States of America,1995
2,False,0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,119050,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,NaN,NaN,en,English,NaN,NaN,US,United States of America,1995
3,False,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN,NaN,NaN,NaN,NaN,NaN,en,English,Twentieth Century Fox Film Corporation,306,US,United States of America,1995
4,False,0,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,96871,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,35,Comedy,en,English,NaN,NaN,US,United States of America,1995


In [41]:
#Convierto los datos de las columnas a formato numerico y le agrego que si por alguna razón encuentra un dato que no se pueda convertir lo setee como NaN y posteriormente esos valores los convierta en 0

df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['revenue'].fillna(0, inplace=True)
df['budget'].fillna(0, inplace=True)

#Creo la columna return como fue solicitada
df['return'] = df.apply(lambda row: 0 if row['budget'] == 0 else row['revenue'] / row['budget'], axis=1)

In [42]:
df.head(2)

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,belongs_to_collection_id,belongs_to_collection_name,belongs_to_collection_poster_path,belongs_to_collection_backdrop_path,genres_id,genres_name,spoken_languages_iso_639_1,spoken_languages_name,production_companies_name,production_companies_id,production_countries_iso_3166_1,production_countries_name,release_year,return
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,NaN,NaN,en,English,Pixar Animation Studios,3,US,United States of America,1995,12.451801
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,United States of America,1995,4.043035


In [44]:
df.to_csv("Data_ML.csv", index=False)

In [45]:
# Elimino las columnas solicitadas
df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'vote_count', 'poster_path', 'homepage'], inplace=True)

In [46]:
#Por ultimo modifico el orden de la columnas
cols = df.columns.tolist()

In [48]:
cols = ['budget','revenue','return',
 'id','title', 'release_date', 'release_year',
 'original_language',
 'overview',
 'popularity',
 'runtime',
 'status',
 'tagline',
 'vote_average',
 'belongs_to_collection_id',
 'belongs_to_collection_name',
 'belongs_to_collection_poster_path',
 'belongs_to_collection_backdrop_path',
 'genres_id',
 'genres_name',
 'spoken_languages_iso_639_1',
 'spoken_languages_name',
 'production_companies_id',
 'production_companies_name',
 'production_countries_iso_3166_1',
 'production_countries_name'
 ]
df = df[cols]

In [49]:
df.head(20)

,budget,revenue,return,id,title,release_date,release_year,original_language,overview,popularity,runtime,status,tagline,vote_average,belongs_to_collection_id,belongs_to_collection_name,belongs_to_collection_poster_path,belongs_to_collection_backdrop_path,genres_id,genres_name,spoken_languages_iso_639_1,spoken_languages_name,production_companies_id,production_companies_name,production_countries_iso_3166_1,production_countries_name
0,30000000,373554033.0,12.451801,862,Toy Story,1995-10-30,1995,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,81.0,Released,NaN,7.7,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,NaN,NaN,en,English,3,Pixar Animation Studios,US,United States of America
1,65000000,262797249.0,4.043035,8844,Jumanji,1995-12-15,1995,en,When siblings Judy and Peter discover an encha...,17.015539,104.0,Released,Roll the dice and unleash the excitement!,6.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,United States of America
2,0,0.0,0.000000,15602,Grumpier Old Men,1995-12-22,1995,en,A family wedding reignites the ancient feud be...,11.7129,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,6.5,119050,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,NaN,NaN,en,English,NaN,NaN,US,United States of America
3,16000000,81452156.0,5.090760,31357,Waiting to Exhale,1995-12-22,1995,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,127.0,Released,Friends are the people who let you be yourself...,6.1,NaN,NaN,NaN,NaN,NaN,NaN,en,English,306,Twentieth Century Fox Film Corporation,US,United States of America
4,0,76578911.0,0.000000,11862,Father of the Bride Part II,1995-02-10,1995,en,Just when George Banks has recovered from his ...,8.387519,106.0,Released,Just When His World Is Back To Normal... He's ...,5.7,96871,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,35,Comedy,en,English,NaN,NaN,US,United States of America
5,60000000,187436818.0,3.123947,949,Heat,1995-12-15,1995,en,"Obsessive master thief, Neil McCauley leads a ...",17.924927,170.0,Released,A Los Angeles Crime Saga,7.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,United States of America
6,58000000,0.0,0.000000,11860,Sabrina,1995-12-15,1995,en,An ugly duckling having undergone a remarkable...,6.677277,127.0,Released,You are cordially invited to the most surprisi...,6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,0.0,0.000000,45325,Tom and Huck,1995-12-22,1995,en,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,97.0,Released,The Original Bad Boys.,5.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Walt Disney Pictures,US,United States of America
8,35000000,64350171.0,1.838576,9091,Sudden Death,1995-12-22,1995,en,International action superstar Jean Claude Van...,5.23158,106.0,Released,Terror goes into overtime.,5.5,NaN,NaN,NaN,NaN,NaN,NaN,en,English,NaN,NaN,US,United States of America
9,58000000,352194034.0,6.072311,710,GoldenEye,1995-11-16,1995,en,James Bond must unmask the mysterious head of ...,14.686036,130.0,Released,No limits. No fears. No substitutes.,6.6,645,James Bond Collection,/HORpg5CSkmeQlAolx3bKMrKgfi.jpg,/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df.to_csv("data_limpia.csv", index=False)
df.to_excel("data_limpia.xlsx", index=False)